In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler

In [2]:
## load data
products = pd.read_csv('cproducts.csv')
tender = pd.read_csv('ctender.csv')

In [3]:
## check shape of files
print('product file has {} rows and {} columns'.format(products.shape[0], products.shape[1]))
print('tender file has {} rows and {} columns'.format(tender.shape[0], tender.shape[1]))

product file has 11972 rows and 16 columns
tender file has 149752 rows and 13 columns


In [4]:
# this data file contains product level information of transactions made by customers
products.head()

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
0,BBID_20482,1975-10-20,male,MADHYA PRADESH,453441.0,2016-05-01,2655,BB-INDORE-MALHAR MEGA MALL,2,85246,NONPROMO,NaN,1000446431,PATANJALI CHOCOS 125g,55.00,Payback
1,BBID_20485,1955-07-21,female,NaN,999999.0,2017-03-09,2655,BB-INDORE-MALHAR MEGA MALL,8,78829,NONPROMO,NaN,1000010431,SOO FRESH ROLLS HOT DOG 4P,30.00,Payback
2,BBID_20498,1981-03-19,female,MAHARASHTRA,400064.0,2015-08-12,2655,BB-INDORE-MALHAR MEGA MALL,13,1064,0000971754,AD 399 FLAT LADIES JEANS FASHION EXC,1000077851005,"DLJ-0010-BSF-2-CJ-FL, 34, BLACK",418.95,Payback
3,BBID_20499,1986-05-10,male,MAHARASHTRA,400099.0,2016-12-18,2615,BB-AMRITSAR-TRILIUM MALL,14,51080,NONPROMO,NaN,1000443231,PATANJALI TOMATO KETCHUP BT 500g,70.00,Payback
4,BBID_204110,1988-02-29,male,MADHYA PRADESH,474001.0,2015-12-31,2655,BB-INDORE-MALHAR MEGA MALL,9,21911,0000974126,NMP 5% OFF CONCEPT DISCOUNT 2655,300481741,MAGGI NDL MERI MASALA PP 70g,34.20,Payback


In [5]:
# this file contains payment mode information used by customers in their transactions
tender.head()

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,tender_type,transaction_number_by_till,payment_amount_by_tender,PaymentUsed
0,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-03,4986,BB-MADURAI-KOCHADAI VILLAGE,22,GVOWN,9208,1000.00,GiftVoucher
1,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-03,4986,BB-MADURAI-KOCHADAI VILLAGE,22,CASH,9208,10.46,CASH
2,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,EDCAXIS,90269,677.28,AXIS
3,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,MOBI,90268,1042.00,One MobiKwik Systems
4,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,MOBI,90269,1000.00,One MobiKwik Systems


### Note:
Let's build a simple model using products file only. We have to create clusters of customers at store level. We'll use k-means algorithm. We'll set value of k as 3 (you can tune and find optimal value)

Clustering process is as follows (work-wise):
1. Convert the data into numeric / float.
2. Since clustering using distance measure, don't forget to scale the data. 
3. Create a matrix /array which will be feeded to the algorithm.
4. Run function.

We'll ensure that sequence of data remains unaffected. Let's see how this is done.

In [6]:
## fill missing values

products['promotion_description'].fillna('no_promo', inplace=True)
products['Gender'].fillna('no_gender', inplace=True)
products['State'].fillna('no_state', inplace=True)
products['PinCode'].fillna(-1, inplace=True)
products['DOB'].fillna("1", inplace=True)

In [7]:
## convert data into numeric / float

for c in products.columns:
    lbl = LabelEncoder()
    if products[c].dtype == 'object' and c not in ['store_description','customerID','transactionDate']:
        products[c] = lbl.fit_transform(products[c])

In [8]:
## scaling, creating matrix and running k-means

stores = list(set(products['store_code']))

cluster_labels = []
cluster_store = []
cluster_data = []
cluster_customers = []
cluster_score = []

for x in stores:
    cld = products[products['store_code'] == x]
    cluster_customers.append(cld['customerID'])
    cld.drop(['store_code','customerID','transactionDate','store_description'], axis=1, inplace=True)
    
    rbs = RobustScaler()
    cld2 = rbs.fit_transform(cld)
    
    n_customers = products['customerID'].loc[products['store_code'] == x].count()
    if n_customers < 2000:
        n_clusters = 4
    else:
        n_clusters = 4
    km1 = KMeans(n_clusters=n_clusters)
    km2 = km1.fit(cld2)
    label = km2.predict(cld2)
    
    s_score = silhouette_score(cld2, label)
    cluster_score.append(s_score)
    
    cluster_labels.append(label)
    cluster_store.append(np.repeat(x, cld.shape[0]))
    cluster_data.append(cld2)

C:\Users\I345351\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
# check mean score per store
np.mean(cluster_score)

0.9337314974847899

In [10]:
## merge list into ndarray
cluster_data = np.concatenate(cluster_data)

In [11]:
## check if the array has same rows as products file - Yes!
cluster_data.shape

(11972L, 12L)

In [12]:
## convert nested lists as 1d array
cluster_customers = np.concatenate(cluster_customers)
cluster_store = np.concatenate(cluster_store)
cluster_labels = np.concatenate(cluster_labels)

In [13]:
## create submission files
sub1 = pd.DataFrame({'customerID':cluster_customers, 'store_code':cluster_store, 'cluster':cluster_labels})

In [14]:
np.savetxt('subOne_18.txt', cluster_data)
sub1.to_csv('subtwo_18.csv', index=False)

#### Next step - go to the folder in your laptop, where these files are saved, zip them and upload on the challenge page and get score.

In [75]:
products.groupby(['store_code'])['customerID'].count()

store_code
2615     723
2655    2767
2906    1257
3692     574
4796    2768
4843    2135
4986    1748
Name: customerID, dtype: int64

In [104]:
products.isnull().sum()

customerID                       0
DOB                            319
Gender                        1028
State                         1583
PinCode                        817
transactionDate                  0
store_code                       0
store_description                0
till_no                          0
transaction_number_by_till       0
promo_code                       0
promotion_description         8510
product_code                     0
product_description              0
sale_price_after_promo           0
discountUsed                     0
dtype: int64

In [130]:
tender.isnull().sum()

customerID                        0
DOB                            4596
Gender                         6663
State                         11650
PinCode                        5564
transactionDate                   0
store_code                        0
store_description                 0
till_no                           0
tender_type                       0
transaction_number_by_till        0
payment_amount_by_tender          0
PaymentUsed                    1223
dtype: int64

In [140]:
products_tender = products.merge(tender, on=['customerID', 'transactionDate', 'store_code', 'till_no', 'transaction_number_by_till'], how='left')

In [141]:
products_tender.head()

,customerID,DOB_x,Gender_x,State_x,PinCode_x,transactionDate,store_code,store_description_x,till_no,transaction_number_by_till,...,sale_price_after_promo,discountUsed,DOB_y,Gender_y,State_y,PinCode_y,store_description_y,tender_type,payment_amount_by_tender,PaymentUsed
0,BBID_20482,1975-10-20,male,MADHYA PRADESH,453441.0,2016-05-01,2655,BB-INDORE-MALHAR MEGA MALL,2,85246,...,55.00,Payback,1975-10-20,male,MADHYA PRADESH,453441.0,BB-INDORE-MALHAR MEGA MALL,EDCUBI,65.00,UBI CREDIT CARD
1,BBID_20485,1955-07-21,female,NaN,999999.0,2017-03-09,2655,BB-INDORE-MALHAR MEGA MALL,8,78829,...,30.00,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BBID_20498,1981-03-19,female,MAHARASHTRA,400064.0,2015-08-12,2655,BB-INDORE-MALHAR MEGA MALL,13,1064,...,418.95,Payback,1981-03-19,female,MAHARASHTRA,400064.0,BB-INDORE-MALHAR MEGA MALL,CASH,768.45,CASH
3,BBID_20499,1986-05-10,male,MAHARASHTRA,400099.0,2016-12-18,2615,BB-AMRITSAR-TRILIUM MALL,14,51080,...,70.00,Payback,1986-05-10,male,MAHARASHTRA,400099.0,BB-AMRITSAR-TRILIUM MALL,MOBI,1701.24,One MobiKwik Systems
4,BBID_204110,1988-02-29,male,MADHYA PRADESH,474001.0,2015-12-31,2655,BB-INDORE-MALHAR MEGA MALL,9,21911,...,34.20,Payback,1988-02-29,male,MADHYA PRADESH,474001.0,BB-INDORE-MALHAR MEGA MALL,CASH,224.89,CASH


In [142]:
products_tender.shape, products.shape, tender.shape

((15605, 24), (11972, 16), (149752, 13))

In [129]:
products['State'].value_counts()

MADHYA PRADESH        3064
KARNATAKA             2373
JHARKHAND             1554
TAMIL NADU            1302
PUNJAB                1156
MAHARASHTRA            174
TAMILNADU              146
WEST BENGAL            106
UTTAR PRADESH          104
DELHI                   60
BIHAR                   49
ORISSA                  47
ANDHRA PRADESH          44
GUJARAT                 35
RAJASTHAN               32
CHATTISGARH             26
HARYANA                 23
KERALA                  20
HARAYANA                14
x                        9
GOA                      9
DUMMY                    9
HIMACHAL PRADESH         6
ASSAM                    5
UTTARANCHAL              4
CHANDIGARH               3
JAMMU AND KASHMIR        2
MADHYA  PRADESH          2
Jharkhand                2
CHHATTISGARH             2
Other                    1
KARNATAK                 1
TRIPURA                  1
MANIPUR                  1
UTTAR PRADESH EAST       1
MP                       1
SIKKIM                   1
N

In [145]:
products['PinCode'].value_counts()

452001.0    1636
580030.0     625
143001.0     584
999999.0     519
452010.0     437
625016.0     369
580024.0     283
831012.0     275
141001.0     268
625010.0     230
831001.0     172
831018.0     146
580020.0     146
580031.0     138
831005.0     133
580023.0     125
831011.0     118
580028.0     100
580032.0      95
452016.0      93
625018.0      91
831004.0      89
452009.0      89
580021.0      87
625001.0      86
832110.0      86
625003.0      70
831009.0      67
452018.0      63
580029.0      62
            ... 
582900.0       1
122015.0       1
600050.0       1
600049.0       1
600045.0       1
600044.0       1
600041.0       1
600040.0       1
600033.0       1
300014.0       1
600023.0       1
600020.0       1
600018.0       1
600014.0       1
813003.0       1
472018.0       1
567201.0       1
390020.0       1
390008.0       1
390007.0       1
590012.0       1
583326.0       1
960091.0       1
583232.0       1
470117.0       1
583121.0       1
422605.0       1
583028.0      

In [146]:
products['till_no'].value_counts()

7     978
6     892
9     809
11    806
4     797
5     763
3     686
13    672
14    649
2     636
10    556
8     553
1     552
12    392
22    363
18    347
15    301
17    301
16    277
19    253
23    176
20    103
25     91
21     17
24      2
Name: till_no, dtype: int64

In [148]:
products['transaction_number_by_till'].value_counts().count()

11127

In [150]:
products['promo_code'].value_counts().count()

2214

In [152]:
products['product_code'].value_counts().count()

6000

In [153]:
products['discountUsed'].value_counts()

Payback                                             6683
T24Club,Payback                                     2085
BBProfitClub                                         998
BBProfitClub,Payback                                 538
FGShoppingFest                                       386
T24Club,BBProfitClub,Payback                         351
BBSavingsClub                                        239
T24Club,BBProfitClub                                 169
FGShoppingFest,Payback                               158
BBSavingsClub,Payback                                107
FGShoppingFest,T24Club,Payback                       104
FGShoppingFest,T24Club                                50
BBSavingsClub,T24Club,Payback                         17
BBSavingsClub,T24Club                                 16
FGShoppingFest,BBProfitClub                           13
FGShoppingFest,T24Club,BBProfitClub,Payback           13
BBSavingsClub,BBProfitClub                            11
FGShoppingFest,BBProfitClub,Pay